# EODHD MCP Server - Portfolio Analysis

This notebook demonstrates advanced portfolio analysis using EODHD data.

## Setup

In [ ]:
import os
import httpx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

API_KEY = os.getenv("EODHD_API_KEY", "demo")
BASE_URL = "https://eodhd.com/api"

def fetch_eod(ticker: str, from_date: str, to_date: str) -> pd.DataFrame:
    """Fetch EOD data for a ticker."""
    url = f"{BASE_URL}/eod/{ticker}"
    params = {
        "api_token": API_KEY,
        "fmt": "json",
        "from": from_date,
        "to": to_date
    }
    response = httpx.get(url, params=params)
    response.raise_for_status()
    df = pd.DataFrame(response.json())
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

## 1. Define Portfolio

In [ ]:
# Define portfolio holdings
portfolio = {
    "AAPL.US": 0.25,   # 25% Apple
    "MSFT.US": 0.25,   # 25% Microsoft
    "GOOGL.US": 0.20,  # 20% Google
    "AMZN.US": 0.15,   # 15% Amazon
    "NVDA.US": 0.15,   # 15% Nvidia
}

# Date range for analysis
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

print(f"Analysis period: {start_date} to {end_date}")
print(f"Portfolio: {list(portfolio.keys())}")

## 2. Fetch Historical Data

In [ ]:
# Fetch data for all tickers
price_data = {}
for ticker in portfolio.keys():
    try:
        df = fetch_eod(ticker, start_date, end_date)
        price_data[ticker] = df['adjusted_close']
        print(f"Fetched {len(df)} days for {ticker}")
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

# Combine into single DataFrame
prices = pd.DataFrame(price_data)
prices = prices.dropna()
print(f"\nTotal trading days: {len(prices)}")
prices.tail()

## 3. Calculate Returns

In [ ]:
# Daily returns
daily_returns = prices.pct_change().dropna()

# Portfolio weighted returns
weights = pd.Series(portfolio)
portfolio_returns = (daily_returns * weights).sum(axis=1)

print("Daily Returns Statistics:")
print(daily_returns.describe())

print(f"\nPortfolio Daily Return (avg): {portfolio_returns.mean():.4%}")
print(f"Portfolio Daily Volatility: {portfolio_returns.std():.4%}")

## 4. Performance Metrics

In [ ]:
# Annualized metrics
trading_days = 252

def calculate_metrics(returns: pd.Series) -> dict:
    """Calculate key performance metrics."""
    ann_return = returns.mean() * trading_days
    ann_volatility = returns.std() * np.sqrt(trading_days)
    sharpe_ratio = ann_return / ann_volatility if ann_volatility > 0 else 0
    
    # Max drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    # Sortino ratio (downside deviation)
    downside_returns = returns[returns < 0]
    downside_std = downside_returns.std() * np.sqrt(trading_days)
    sortino_ratio = ann_return / downside_std if downside_std > 0 else 0
    
    return {
        "Annual Return": f"{ann_return:.2%}",
        "Annual Volatility": f"{ann_volatility:.2%}",
        "Sharpe Ratio": f"{sharpe_ratio:.2f}",
        "Sortino Ratio": f"{sortino_ratio:.2f}",
        "Max Drawdown": f"{max_drawdown:.2%}",
    }

# Portfolio metrics
metrics = calculate_metrics(portfolio_returns)
print("Portfolio Performance Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

## 5. Individual Stock Analysis

In [ ]:
# Calculate metrics for each stock
stock_metrics = {}
for ticker in portfolio.keys():
    if ticker in daily_returns.columns:
        stock_metrics[ticker] = calculate_metrics(daily_returns[ticker])

# Display as DataFrame
metrics_df = pd.DataFrame(stock_metrics).T
print("Individual Stock Metrics:")
metrics_df

## 6. Correlation Analysis

In [ ]:
# Correlation matrix
correlation = daily_returns.corr()
print("Correlation Matrix:")
correlation.round(3)

## 7. Cumulative Returns

In [ ]:
# Calculate cumulative returns
cumulative_returns = (1 + daily_returns).cumprod() - 1
portfolio_cumulative = (1 + portfolio_returns).cumprod() - 1

print("Total Returns:")
for ticker in portfolio.keys():
    if ticker in cumulative_returns.columns:
        total_return = cumulative_returns[ticker].iloc[-1]
        print(f"  {ticker}: {total_return:.2%}")

print(f"\n  Portfolio: {portfolio_cumulative.iloc[-1]:.2%}")

## Summary

This notebook demonstrated:

1. Building a weighted portfolio
2. Fetching historical data for multiple tickers
3. Calculating daily and portfolio returns
4. Computing performance metrics (Sharpe, Sortino, Max Drawdown)
5. Correlation analysis between holdings
6. Cumulative return calculation

For visualization, consider adding matplotlib/plotly charts for:
- Cumulative returns comparison
- Correlation heatmap
- Drawdown chart
- Rolling metrics